In [2]:
from google.colab import files

uploaded=files.upload()
for fn in uploaded.keys():
    print('user uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))



Saving image.zip to image.zip
user uploaded file "image.zip" with length 1919308 bytes


In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
!ls


datalab  image.zip


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('image.zip', 'r')
zip_ref.extractall()
zip_ref.close()


In [5]:
from scipy.misc import imread
from scipy.misc import imresize
import os
ls=[]
folder='wwe'
files=os.listdir(folder)
files=list(map(lambda x: os.path.join(folder,x),files))
a=(len(files))
for i in range(a):
    im = imread(files[i])
    im=imresize(im,[32,32,3])
    im=im.flatten()
    ls.append(im)


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [6]:
import numpy as np

raw_data=np.array(ls,dtype=object)
print(raw_data.shape)

(2370, 3072)


In [7]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(raw_data,train_size=0.7,test_size=0.3,shuffle=True)
print(train_data.shape)

(1659, 3072)


In [8]:
(x_train, y_train), (x_test, y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 7s 0us/step


In [9]:
x1=x_train[0:10000,:]
x2=x_train[10000:20000,:]
x1.shape,x2.shape

((10000, 32, 32, 3), (10000, 32, 32, 3))

In [10]:
x1=x1.reshape(10000,3072)
x2=x2.reshape(10000,3072)
x1.shape,x2.shape

((10000, 3072), (10000, 3072))

In [11]:
y1=y_train[0:10000]
y2=y_train[10000:20000]
y1.shape,y2.shape


((10000, 1), (10000, 1))

In [12]:
beg=10000
end=len(x1) + len(train_data)
#print(end)
images=np.zeros(shape=[end+1,3072])
images[0:beg, :]=x1
images[beg:end,:]=train_data
images[end]=images[11330]
print(images.shape)

(11660, 3072)


In [0]:
images=images.reshape(-1,32,32,3)
images=images/255.0
#print(images[2])

In [14]:
images.shape

(11660, 32, 32, 3)

In [15]:
tes=x2[0:5000,:]
print(tes.shape)
end=len(tes) + len(test_data)
test_images=np.zeros(shape=[end,3072])
beg=5000
test_images[0:beg,:]=tes
test_images[beg:end,:]=test_data
print(test_images[5201])
print(test_images.shape)

(5000, 3072)
[ 52.  54.  77. ... 240. 238. 213.]
(5711, 3072)


In [16]:
test_images=test_images.reshape(-1,32,32,3)
print(test_images.shape)
test_images=test_images/255.0

(5711, 32, 32, 3)


In [17]:
test_images.shape

(5711, 32, 32, 3)

In [18]:
label_train=np.zeros(shape=[11660,1])
label_train[10000:11660,:]=1
print(label_train.shape)

(11660, 1)


In [19]:
label_test=np.zeros(shape=[5711,1])
label_test[5000:5711,:]=1
print(label_test[5001])

[1.]


In [20]:
a=(images.shape[0])
i=np.random.permutation(a)
x_train1,y_train1=images[i],label_train[i]
print(y_train1.shape,x_train1.shape)

(11660, 1) (11660, 32, 32, 3)


In [21]:
j=np.random.permutation(test_images.shape[0])
data_test,label_test=test_images[j],label_test[j]
print(data_test.shape,label_test.shape)

(5711, 32, 32, 3) (5711, 1)


In [0]:
num_classes=2
y_train_main = keras.utils.to_categorical(y_train1, num_classes)
y_test_main = keras.utils.to_categorical(label_test, num_classes)


In [23]:
y_train_main.shape,y_test_main.shape

((11660, 2), (5711, 2))

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train1.shape[1:]))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [0]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [26]:
batch_size=32
epochs=8
model.fit(x_train1, y_train_main,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(data_test, y_test_main),
              shuffle=True)

Train on 11660 samples, validate on 5711 samples
Epoch 1/8
11660/11660 [==============================] - 10s 818us/step - loss: 0.1556 - acc: 0.9407 - val_loss: 0.0647 - val_acc: 0.9746
Epoch 2/8
11660/11660 [==============================] - 7s 600us/step - loss: 0.0737 - acc: 0.9730 - val_loss: 0.0498 - val_acc: 0.9800
Epoch 3/8
 1184/11660 [==>...........................] - ETA: 5s - loss: 0.0618 - acc: 0.9738

11660/11660 [==============================] - 7s 594us/step - loss: 0.0609 - acc: 0.9770 - val_loss: 0.1051 - val_acc: 0.9545
Epoch 4/8
11660/11660 [==============================] - 7s 592us/step - loss: 0.0519 - acc: 0.9804 - val_loss: 0.0343 - val_acc: 0.9872
Epoch 5/8
11660/11660 [==============================] - 7s 585us/step - loss: 0.0424 - acc: 0.9850 - val_loss: 0.0311 - val_acc: 0.9881
Epoch 6/8
 7296/11660 [=================>............] - ETA: 2s - loss: 0.0371 - acc: 0.9864

11660/11660 [==============================] - 7s 584us/step - loss: 0.0386 - acc: 0.9856 - val_loss: 0.0269 - val_acc: 0.9925
Epoch 7/8
11660/11660 [==============================] - 7s 588us/step - loss: 0.0334 - acc: 0.9866 - val_loss: 0.0287 - val_acc: 0.9904
Epoch 8/8
11660/11660 [==============================] - 7s 586us/step - loss: 0.0314 - acc: 0.9886 - val_loss: 0.0296 - val_acc: 0.9893


In [27]:
scores = model.evaluate(data_test, y_test_main, verbose=1)
scores


5711/5711 [==============================] - 1s 157us/step


[0.029612501269640837, 0.9893188583539844]

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [29]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [31]:
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [37]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(data_test, y_test_main, verbose=1)

5711/5711 [==============================] - 1s 183us/step


In [38]:
score

[0.029612501269640837, 0.9893188583539844]

In [0]:
from google.colab import files

In [0]:
files.download("model.h5")

In [0]:
files.download("model.json")